# Booking.com Reviews — NLP Multilíngue (Estudo de Caso)
**Foco:** comparar padrões de reviews em inglês, alemão e francês.

Este notebook apresenta um estudo de caso com linguagem executiva: transformamos avaliações
textuais em sinais objetivos que suportam decisões operacionais e de comunicação.


## Contexto de negócio
Hotéis recebem reviews de hóspedes em diferentes idiomas. A análise agregada pode mascarar
diferenças culturais ou de expectativa. Ao segmentar por idioma, obtemos um retrato mais
fiel do que gera satisfação e insatisfação.


## Decisão suportada
- Priorizar melhorias de serviço com base nos temas negativos mais recorrentes por idioma.
- Ajustar mensagens de marketing de acordo com o que cada grupo valoriza.


## Dados e escopo
- Fonte: dataset de reviews (Booking.com)
- Campos principais: `reviewTextParts/Liked`, `reviewTextParts/Disliked`, `reviewOriginalLanguage`
- Idiomas-alvo: EN, DE, FR


## Metodologia (resumo)
1. Normalização de texto preservando acentos
2. Detecção de idioma (campo original + fallback por stopwords)
3. Lemmatização com spaCy por idioma
4. Filtro de POS (ADJ/NOUN/PROPN)
5. Extração de trigramas por idioma e sentimento


In [ ]:
from pathlib import Path
import pandas as pd

from src.pipeline import load_config, run_pipeline
from src.report import generate_report


In [ ]:
# Selecione quais relatorios deseja gerar
def choose_config_paths():
    print("Escolha o relatorio a gerar:")
    print("1 - Berlin")
    print("2 - Creta")
    print("3 - Todos")
    choice = input("Digite 1, 2 ou 3: ").strip()
    mapping = {
        "1": ["config/config_berlin.json"],
        "2": ["config/config_crete.json"],
        "3": ["config/config_berlin.json", "config/config_crete.json"],
    }
    return mapping.get(choice, mapping["3"])

config_paths = choose_config_paths()


### Observação
Se for a primeira execução, baixe os modelos do spaCy:
- `python -m spacy download en_core_web_sm`
- `python -m spacy download de_core_news_sm`
- `python -m spacy download fr_core_news_sm`


In [ ]:
results_map = {}
for path in config_paths:
    config = load_config(path)
    results_map[path] = run_pipeline(config, write_outputs=True)
    report_text = generate_report(config)
    report_path = Path(config["report_path"])
    report_path.parent.mkdir(parents=True, exist_ok=True)
    report_path.write_text(report_text, encoding="utf-8")
    print(f"Report gerado: {report_path}")

active_config = config_paths[0]
print(f"Exibindo resultados de: {active_config}")
results = results_map[active_config]
results["counts"]


In [ ]:
top = results["top_ngrams"]
top.query("language == 'en' and sentiment == 'good'").head(10)


In [ ]:
import matplotlib.pyplot as plt

counts = results["counts"]
pivot = counts.pivot(index="language", columns="sentiment", values="reviews").fillna(0)
pivot.plot(kind="bar", figsize=(8, 4), title="Volume de reviews por idioma e sentimento")
plt.tight_layout()


## Conclusões e recomendações
- Use os temas negativos recorrentes para priorizar melhorias operacionais.
- Compare diferenças entre idiomas para calibrar comunicação e expectativa do hóspede.
- Utilize os temas positivos como prova social segmentada por mercado.
